P.S: USING LAB04 CODES TO GENERATE A BETTER MODEL ON MY PREVIOUS ASSIGNMENT WORKS




# **BEST SETTINGS THAT I'VE FOUND IN MY WORK ~ 87% (86.99%)**

*BATCH_SIZE* = 1621 \\
AT *ITERATION* 14500 \\
USING 5 *HIDDEN LAYERS* (each having 300 nodes, and *LeakyRELU* as activation layer)
*Learining rate* was 0.1

# **ASSIGNMENT 02**
## **PROBLEM 1 :**
#### **Apply only logistic regression for the  NumtaDB dataset and build a multiclass classification model that can recognize [0-9] Bengali handwritten digits with different hyperparameter settings.**

In [ ]:
'''
Name : Rahat Bin Osman
ID: 160204083
Group: B1
'''

'\nName : Rahat Bin Osman\nID: 160204083\nGroup: B1\n'

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle (1).json


In [ ]:
!kaggle datasets download -d BengaliAI/numta

numta.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import os 
import zipfile 
import matplotlib.pyplot as plt
import pandas as pd
import shutil 
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.data import Dataset
from torchvision import datasets, transforms, models
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dsets

In [ ]:
local_dir='/content/numta.zip'
zip_ref=zipfile.ZipFile(local_dir,'r')
zip_ref.extractall('/tmp')
zip_ref.close

<bound method ZipFile.close of <zipfile.ZipFile filename='/content/numta.zip' mode='r'>>

In [ ]:
PATH = '/tmp/'
os.listdir(PATH)

['dap_multiplexer.68ca039041f2.root.log.INFO.20210306-183207.47',
 'training-e',
 'testing-d',
 'training-a.csv',
 'training-e.csv',
 'testing-all-corrected',
 'training-c',
 'training-d.csv',
 'training-d',
 'testing-e',
 'testing-auga',
 'training-b.csv',
 'train',
 'dap_multiplexer.INFO',
 'debugger_1rt0lxs13m',
 'initgoogle_syslog_dir.0',
 'testing-augc',
 'testing-c',
 'testing-f',
 'testing-b',
 'training-c.csv',
 'training-b',
 'testing-a',
 'training-a']

In [ ]:
df=pd.read_csv("/tmp/training-a.csv")

In [ ]:
df.columns.values

array(['filename', 'original filename', 'scanid', 'digit',
       'database name original', 'contributing team', 'database name'],
      dtype=object)

In [ ]:
col_list = ["filename", "digit"]

df1=pd.read_csv("/tmp/training-a.csv",usecols=col_list)
df2=pd.read_csv("/tmp/training-b.csv",usecols=col_list)
df3=pd.read_csv("/tmp/training-c.csv",usecols=col_list)
df4=pd.read_csv("/tmp/training-d.csv",usecols=col_list)
df5=pd.read_csv("/tmp/training-e.csv",usecols=col_list)

In [ ]:
dataframes=[df1,df2,df3,df4,df5]
db = pd.concat(dataframes, ignore_index=True)

**DATASET IS READY TO USE WITH A TOTAL OF 72045 IMAGES**

> WHERE 90% will go for training and the rest 10% will be used for testing.



In [ ]:
db.info

<bound method DataFrame.info of          filename  digit
0      a00000.png      5
1      a00001.png      3
2      a00002.png      1
3      a00003.png      7
4      a00004.png      0
...           ...    ...
72040  e16773.png      2
72041  e16774.png      6
72042  e16775.png      5
72043  e16776.png      4
72044  e16777.png      4

[72045 rows x 2 columns]>

In [ ]:
if not os.path.exists("/tmp/train"):
    TRAIN_PATH=os.mkdir("/tmp/train")

ls1=os.listdir("/tmp/training-a")
ls2=os.listdir("/tmp/training-b")
ls3=os.listdir("/tmp/training-c")
ls4=os.listdir("/tmp/training-d")
ls5=os.listdir("/tmp/training-e")

src="/tmp/training-a"
TRAIN_PATH="/tmp/train"
for image in ls1:
  file_name = os.path.join(src, image)
  if os.path.isfile(file_name):
   shutil.copy(file_name, TRAIN_PATH)  

src="/tmp/training-b"
TRAIN_PATH="/tmp/train"
for image in ls2:
  file_name = os.path.join(src, image)
  if os.path.isfile(file_name):
   shutil.copy(file_name, TRAIN_PATH)  

src="/tmp/training-c"
TRAIN_PATH="/tmp/train"
for image in ls3:
  file_name = os.path.join(src, image)
  if os.path.isfile(file_name):
   shutil.copy(file_name, TRAIN_PATH)  

src="/tmp/training-d"
TRAIN_PATH="/tmp/train"
for image in ls4:
  file_name = os.path.join(src, image)
  if os.path.isfile(file_name):
   shutil.copy(file_name, TRAIN_PATH)  

src="/tmp/training-e"
TRAIN_PATH="/tmp/train"
for image in ls5:
  file_name = os.path.join(src, image)
  if os.path.isfile(file_name):
   shutil.copy(file_name, TRAIN_PATH)  

In [ ]:
print(len(os.listdir("/tmp/train")))
print(len(db))

72045
72045


In [ ]:
class Dataset(Dataset):
    def __init__(self, df, root, transform=None):
        self.data = df
        self.root = root
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        item = self.data.iloc[index]
        path = self.root + "/" + item[0]
        image = Image.open(path).convert('L')
        label = item[1]
        
        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [ ]:
mean = [0.5] 
std = [0.5]

transform = transforms.Compose([
    transforms.Resize((28,28)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

train_data  = Dataset(db, TRAIN_PATH, transform)

print("Training Samples:",len(train_data))

Training Samples: 72045


In [ ]:
train_dataset, test_dataset = train_test_split(train_data, test_size = 0.1)
print(len(train_dataset))
print(len(test_dataset))

64840
7205


# **SETTING 01**


In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets

# Hyperparameters
batch_size = 100
num_iters = 30000
input_dim = 28*28 #num_features = 784
num_hidden = 100
output_dim = 10

learning_rate = 0.1

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

num_epochs = num_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)  

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

class DeepNeuralNetworkModel(nn.Module):
    def __init__(self, input_size, num_classes, num_hidden):
        super().__init__()
        ### 1st hidden layer: 784 --> 100
        self.linear_1 = nn.Linear(input_size, num_hidden)
        ### Non-linearity in 1st hidden layer
        self.relu_1 = nn.ReLU()

        ### 2nd hidden layer: 100 --> 100
        self.linear_2 = nn.Linear(num_hidden, num_hidden)
        ### Non-linearity in 2nd hidden layer
        self.relu_2 = nn.ReLU()

        ### 3rd hidden layer: 100 --> 100
        self.linear_3 = nn.Linear(num_hidden, num_hidden)
        ### Non-linearity in 3rd hidden layer
        self.relu_3 = nn.ReLU()

        ### Output layer: 100 --> 10
        self.linear_out = nn.Linear(num_hidden, num_classes)

    def forward(self, x):
        ### 1st hidden layer
        out  = self.linear_1(x)
        ### Non-linearity in 1st hidden layer
        out = self.relu_1(out)
        
        ### 2nd hidden layer
        out  = self.linear_2(out)
        ### Non-linearity in 2nd hidden layer
        out = self.relu_2(out)

        ### 3rd hidden layer
        out  = self.linear_3(out)
        ### Non-linearity in 3rd hidden layer
        out = self.relu_3(out)
        
        # Linear layer (output)
        probas  = self.linear_out(out)
        return probas

# INSTANTIATE MODEL CLASS

model = DeepNeuralNetworkModel(input_size = input_dim,
                               num_classes = output_dim,
                               num_hidden = num_hidden)
# To enable GPU
model.to(device)

# INSTANTIATE LOSS & OPTIMIZER CLASS
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):

        images = images.view(-1, 28*28).to(device)
        labels = labels.to(device)

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = model(images) 

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
               
                images = images.view(-1, 28*28).to(device)

                # Forward pass only to get logits/output
                outputs = model(images)

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs, 1)

                # Total number of labels
                total += labels.size(0)


                # Total correct predictions
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum() 
                else:
                    correct += (predicted == labels).sum()

            accuracy = 100 * correct.item() / total

            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 500. Loss: 1.9079991579055786. Accuracy: 30.11797362942401
Iteration: 1000. Loss: 1.805791974067688. Accuracy: 31.43650242886884
Iteration: 1500. Loss: 2.032890796661377. Accuracy: 29.22970159611381
Iteration: 2000. Loss: 1.8418611288070679. Accuracy: 31.658570437196392
Iteration: 2500. Loss: 1.9723045825958252. Accuracy: 33.96252602359473
Iteration: 3000. Loss: 1.732933521270752. Accuracy: 42.664816099930604
Iteration: 3500. Loss: 1.7209173440933228. Accuracy: 39.19500346981263
Iteration: 4000. Loss: 1.5846437215805054. Accuracy: 43.78903539208883
Iteration: 4500. Loss: 1.629274845123291. Accuracy: 49.673837612768914
Iteration: 5000. Loss: 1.5521982908248901. Accuracy: 40.360860513532266
Iteration: 5500. Loss: 1.4526848793029785. Accuracy: 46.05135322692575
Iteration: 6000. Loss: 1.457910180091858. Accuracy: 46.67591950034698
Iteration: 6500. Loss: 1.349843144416809. Accuracy: 46.16238723108952
Iteration: 7000. Loss: 1.3347256183624268. Accuracy: 55.475364330326165
Iteratio

# **SETTING 02**


In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets

# Hyperparameters
batch_size = 1621
num_iters = 12000
input_dim = 28*28 #num_features = 784
num_hidden = 100
output_dim = 10

learning_rate = 0.1

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

num_epochs = num_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)  

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

class DeepNeuralNetworkModel(nn.Module):
    def __init__(self, input_size, num_classes, num_hidden):
        super().__init__()
        ### 1st hidden layer: 784 --> 100
        self.linear_1 = nn.Linear(input_size, num_hidden)
        ### Non-linearity in 1st hidden layer
        self.relu_1 = nn.LeakyReLU()

        ### 2nd hidden layer: 100 --> 100
        self.linear_2 = nn.Linear(num_hidden, num_hidden)
        ### Non-linearity in 2nd hidden layer
        self.relu_2 = nn.LeakyReLU()

        ### 3rd hidden layer: 100 --> 100
        self.linear_3 = nn.Linear(num_hidden, num_hidden)
        ### Non-linearity in 3rd hidden layer
        self.relu_3 = nn.LeakyReLU()

        ### 4th hidden layer: 100 --> 100
        self.linear_4 = nn.Linear(num_hidden, num_hidden)
        ### Non-linearity in 3rd hidden layer
        self.relu_4 = nn.LeakyReLU()

        ### Output layer: 100 --> 10
        self.linear_out = nn.Linear(num_hidden, num_classes)

    def forward(self, x):
        ### 1st hidden layer
        out  = self.linear_1(x)
        ### Non-linearity in 1st hidden layer
        out = self.relu_1(out)
        
        ### 2nd hidden layer
        out  = self.linear_2(out)
        ### Non-linearity in 2nd hidden layer
        out = self.relu_2(out)

        ### 3rd hidden layer
        out  = self.linear_3(out)
        ### Non-linearity in 3rd hidden layer
        out = self.relu_3(out)

         ### 4th hidden layer
        out  = self.linear_4(out)
        ### Non-linearity in 3rd hidden layer
        out = self.relu_4(out)
        
        # Linear layer (output)
        probas  = self.linear_out(out)
        return probas

# INSTANTIATE MODEL CLASS

model = DeepNeuralNetworkModel(input_size = input_dim,
                               num_classes = output_dim,
                               num_hidden = num_hidden)
# To enable GPU
model.to(device)

# INSTANTIATE LOSS & OPTIMIZER CLASS
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):

        images = images.view(-1, 28*28).to(device)
        labels = labels.to(device)

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = model(images) 

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
               
                images = images.view(-1, 28*28).to(device)

                # Forward pass only to get logits/output
                outputs = model(images)

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs, 1)

                # Total number of labels
                total += labels.size(0)


                # Total correct predictions
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum() 
                else:
                    correct += (predicted == labels).sum()

            accuracy = 100 * correct.item() / total

            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 500. Loss: 2.0142323970794678. Accuracy: 23.913948646773076
Iteration: 1000. Loss: 1.8226474523544312. Accuracy: 30.97848716169327
Iteration: 1500. Loss: 1.7255643606185913. Accuracy: 33.088133240805
Iteration: 2000. Loss: 1.7743223905563354. Accuracy: 38.6675919500347
Iteration: 2500. Loss: 1.654906988143921. Accuracy: 46.717557251908396
Iteration: 3000. Loss: 1.6134486198425293. Accuracy: 37.65440666204025
Iteration: 3500. Loss: 1.6101983785629272. Accuracy: 45.53782095766829
Iteration: 4000. Loss: 1.4461123943328857. Accuracy: 51.33934767522554
Iteration: 4500. Loss: 1.606313705444336. Accuracy: 43.59472588480222
Iteration: 5000. Loss: 1.261786699295044. Accuracy: 50.978487161693266
Iteration: 5500. Loss: 1.3908493518829346. Accuracy: 51.36710617626648
Iteration: 6000. Loss: 1.2241040468215942. Accuracy: 47.369882026370576
Iteration: 6500. Loss: 1.3232873678207397. Accuracy: 59.916724496877166
Iteration: 7000. Loss: 1.1154199838638306. Accuracy: 49.49340735600278
Iteratio

# **SETTING 03**


In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets

# Hyperparameters
batch_size = 1621
num_iters = 12000
input_dim = 28*28 #num_features = 784
num_hidden = 300
output_dim = 10

learning_rate = 0.1

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

num_epochs = num_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)  

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

class DeepNeuralNetworkModel(nn.Module):
    def __init__(self, input_size, num_classes, num_hidden):
        super().__init__()
        ### 1st hidden layer: 784 --> 100
        self.linear_1 = nn.Linear(input_size, num_hidden)
        ### Non-linearity in 1st hidden layer
        self.relu_1 = nn.LeakyReLU()

        ### 2nd hidden layer: 100 --> 100
        self.linear_2 = nn.Linear(num_hidden, num_hidden)
        ### Non-linearity in 2nd hidden layer
        self.relu_2 = nn.LeakyReLU()

        ### 3rd hidden layer: 100 --> 100
        self.linear_3 = nn.Linear(num_hidden, num_hidden)
        ### Non-linearity in 3rd hidden layer
        self.relu_3 = nn.LeakyReLU()

        ### 4th hidden layer: 100 --> 100
        self.linear_4 = nn.Linear(num_hidden, num_hidden)
        ### Non-linearity in 3rd hidden layer
        self.relu_4 = nn.LeakyReLU()

        ### 5th hidden layer: 100 --> 100
        self.linear_5 = nn.Linear(num_hidden, num_hidden)
        ### Non-linearity in 3rd hidden layer
        self.relu_5 = nn.LeakyReLU()

        ### Output layer: 100 --> 10
        self.linear_out = nn.Linear(num_hidden, num_classes)

    def forward(self, x):
        ### 1st hidden layer
        out  = self.linear_1(x)
        ### Non-linearity in 1st hidden layer
        out = self.relu_1(out)
        
        ### 2nd hidden layer
        out  = self.linear_2(out)
        ### Non-linearity in 2nd hidden layer
        out = self.relu_2(out)

        ### 3rd hidden layer
        out  = self.linear_3(out)
        ### Non-linearity in 3rd hidden layer
        out = self.relu_3(out)

         ### 4th hidden layer
        out  = self.linear_4(out)
        ### Non-linearity in 3rd hidden layer
        out = self.relu_4(out)

        ### 5th hidden layer
        out  = self.linear_5(out)
        ### Non-linearity in 3rd hidden layer
        out = self.relu_5(out)
        
        # Linear layer (output)
        probas  = self.linear_out(out)
        return probas

# INSTANTIATE MODEL CLASS

model = DeepNeuralNetworkModel(input_size = input_dim,
                               num_classes = output_dim,
                               num_hidden = num_hidden)
# To enable GPU
model.to(device)

# INSTANTIATE LOSS & OPTIMIZER CLASS
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):

        images = images.view(-1, 28*28).to(device)
        labels = labels.to(device)

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = model(images) 

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
               
                images = images.view(-1, 28*28).to(device)

                # Forward pass only to get logits/output
                outputs = model(images)

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs, 1)

                # Total number of labels
                total += labels.size(0)


                # Total correct predictions
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum() 
                else:
                    correct += (predicted == labels).sum()

            accuracy = 100 * correct.item() / total

            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 500. Loss: 2.279655694961548. Accuracy: 15.794587092297016
Iteration: 1000. Loss: 1.9512073993682861. Accuracy: 28.57737682165163
Iteration: 1500. Loss: 1.783829927444458. Accuracy: 34.823039555863986
Iteration: 2000. Loss: 1.6334381103515625. Accuracy: 36.28036086051353
Iteration: 2500. Loss: 1.6513386964797974. Accuracy: 31.700208188757806
Iteration: 3000. Loss: 1.6295095682144165. Accuracy: 38.986814712005554
Iteration: 3500. Loss: 1.5966248512268066. Accuracy: 38.00138792505205
Iteration: 4000. Loss: 1.4578416347503662. Accuracy: 51.117279666897986
Iteration: 4500. Loss: 1.395443081855774. Accuracy: 49.09090909090909
Iteration: 5000. Loss: 1.5464473962783813. Accuracy: 45.05204718945177
Iteration: 5500. Loss: 1.729675054550171. Accuracy: 45.45454545454545
Iteration: 6000. Loss: 1.3096617460250854. Accuracy: 54.54545454545455
Iteration: 6500. Loss: 1.2352452278137207. Accuracy: 58.14018043025676
Iteration: 7000. Loss: 1.2660412788391113. Accuracy: 50.298403886190144
Itera

# **SETTING 04**


# **BEST SETTINGS THAT I'VE FOUND IN MY WORK ~ 87% (86.99%)**

*BATCH_SIZE* = 1621 \\
AT *ITERATION* 14500 \\
USING 5 *HIDDEN LAYERS* (each having 300 nodes, and *LeakyRELU* as activation layer)
*Learining rate* was 0.1

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets

# Hyperparameters
batch_size = 1621
num_iters = 15000
input_dim = 28*28 #num_features = 784
num_hidden = 300
output_dim = 10

learning_rate = 0.1

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

num_epochs = num_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)  

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

class DeepNeuralNetworkModel(nn.Module):
    def __init__(self, input_size, num_classes, num_hidden):
        super().__init__()
        ### 1st hidden layer: 784 --> 100
        self.linear_1 = nn.Linear(input_size, num_hidden)
        ### Non-linearity in 1st hidden layer
        self.relu_1 = nn.LeakyReLU()

        ### 2nd hidden layer: 100 --> 100
        self.linear_2 = nn.Linear(num_hidden, num_hidden)
        ### Non-linearity in 2nd hidden layer
        self.relu_2 = nn.LeakyReLU()

        ### 3rd hidden layer: 100 --> 100
        self.linear_3 = nn.Linear(num_hidden, num_hidden)
        ### Non-linearity in 3rd hidden layer
        self.relu_3 = nn.LeakyReLU()

        ### 4th hidden layer: 100 --> 100
        self.linear_4 = nn.Linear(num_hidden, num_hidden)
        ### Non-linearity in 3rd hidden layer
        self.relu_4 = nn.LeakyReLU()

        ### 5th hidden layer: 100 --> 100
        self.linear_5 = nn.Linear(num_hidden, num_hidden)
        ### Non-linearity in 3rd hidden layer
        self.relu_5 = nn.LeakyReLU()

        ### Output layer: 100 --> 10
        self.linear_out = nn.Linear(num_hidden, num_classes)

    def forward(self, x):
        ### 1st hidden layer
        out  = self.linear_1(x)
        ### Non-linearity in 1st hidden layer
        out = self.relu_1(out)
        
        ### 2nd hidden layer
        out  = self.linear_2(out)
        ### Non-linearity in 2nd hidden layer
        out = self.relu_2(out)

        ### 3rd hidden layer
        out  = self.linear_3(out)
        ### Non-linearity in 3rd hidden layer
        out = self.relu_3(out)

         ### 4th hidden layer
        out  = self.linear_4(out)
        ### Non-linearity in 3rd hidden layer
        out = self.relu_4(out)

        ### 5th hidden layer
        out  = self.linear_5(out)
        ### Non-linearity in 3rd hidden layer
        out = self.relu_5(out)
        
        # Linear layer (output)
        probas  = self.linear_out(out)
        return probas

# INSTANTIATE MODEL CLASS

model = DeepNeuralNetworkModel(input_size = input_dim,
                               num_classes = output_dim,
                               num_hidden = num_hidden)
# To enable GPU
model.to(device)

# INSTANTIATE LOSS & OPTIMIZER CLASS
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):

        images = images.view(-1, 28*28).to(device)
        labels = labels.to(device)

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = model(images) 

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
               
                images = images.view(-1, 28*28).to(device)

                # Forward pass only to get logits/output
                outputs = model(images)

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs, 1)

                # Total number of labels
                total += labels.size(0)


                # Total correct predictions
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum() 
                else:
                    correct += (predicted == labels).sum()

            accuracy = 100 * correct.item() / total

            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 500. Loss: 2.2745096683502197. Accuracy: 21.17973629424011
Iteration: 1000. Loss: 1.8771384954452515. Accuracy: 27.300485773768216
Iteration: 1500. Loss: 1.8066753149032593. Accuracy: 27.605829285218597
Iteration: 2000. Loss: 1.6924033164978027. Accuracy: 40.41637751561416
Iteration: 2500. Loss: 1.7915600538253784. Accuracy: 34.26786953504511
Iteration: 3000. Loss: 1.5187245607376099. Accuracy: 44.066620402498266
Iteration: 3500. Loss: 1.5696516036987305. Accuracy: 50.38167938931298
Iteration: 4000. Loss: 1.4078588485717773. Accuracy: 48.2442748091603
Iteration: 4500. Loss: 1.329304575920105. Accuracy: 50.58986814712006
Iteration: 5000. Loss: 1.4561400413513184. Accuracy: 50.367800138792504
Iteration: 5500. Loss: 1.4135366678237915. Accuracy: 46.95350451075642
Iteration: 6000. Loss: 1.3669509887695312. Accuracy: 59.083969465648856
Iteration: 6500. Loss: 0.9928193688392639. Accuracy: 58.94517696044414
Iteration: 7000. Loss: 1.4341951608657837. Accuracy: 46.12074947952811
Iter

# **PREDICTIONS IN TEST DATAS**
### **(45% ACCURACY MODEL)**
- We used the 5th model that had the following settings:
  - batch_size = 1621 
  - num_iters = 30000
  - num_features = 784
  - output_dim = 10
  - learning_rate = 0.003

